# Basics
The basic notebook just imports the required dependencies and sets the ports and addresses correctly to be accessible outside of the Docker container. It then creates a link for viewing the data

In [ ]:
from __future__ import print_function
from neuroglancer.server import global_server_args
# setup the binding so it is visible outside the container
PORT = 8989
global_server_args['bind_port'] = PORT
global_server_args['bind_address'] = '00.0.0.0' # TODO: the server renames addresses of 0.0.0.0 and they become unusable
import neuroglancer
import numpy as np

# Synthetic Data
Since it is much easier to generate synthetic data than copy full real images a 3D image sized 100x100x100 is greated and then colored using a simple sinusoidal relationship

In [ ]:
a = np.zeros((3, 100, 100, 100), dtype=np.uint8)
ix, iy, iz = np.meshgrid(*[np.linspace(0, 1, n) for n in a.shape[1:]], indexing='ij')
a[0, :, :, :] = np.abs(np.sin(4 * (ix + iy))) * 255
a[1, :, :, :] = np.abs(np.sin(4 * (iy + iz))) * 255
a[2, :, :, :] = np.abs(np.sin(4 * (ix + iz))) * 255

b = np.cast[np.uint32](np.floor(np.sqrt((ix - 0.5)**2 + (iy - 0.5)**2 + (iz - 0.5)**2) * 10))
b = np.pad(b, 1, 'constant')
print(a.shape, b.shape)

# Create the viewer
Here the viewer is created with a simple shader to make the rendering colorful

In [ ]:
neuroglancer.set_static_content_source(url='https://neuroglancer-demo.appspot.com')

viewer = neuroglancer.Viewer()
a_vol = neuroglancer.LocalVolume(a, offset=(200, 300, 150))
viewer.volume_manager.register_volume(a_vol)
#viewer.add(b, name='b')

# Show the Results
Here when we show the viewer we get a link, altnatively the results can be shown using the IPython HTML displaying interface

In [ ]:
import os
in_binder ='JUPYTERHUB_SERVICE_PREFIX' in os.environ
if in_binder:
    base_prefix = '{}proxy/{}/'.format(os.environ['JUPYTERHUB_SERVICE_PREFIX'], PORT)
    url = 'https://hub.mybinder.org{}'.format(base_prefix)
else:
    url = 'http://localhost:%d' % port
    base_prefix = ''

In [ ]:
from IPython.display import HTML

new_url = '{}/v/{}'.format(url, viewer.get_viewer_url().split('/v/')[-1])

HTML(data = '<a href="{url}" target="_new">Open in new window</a><hr><iframe src="{url}" width = "720px" height = "400px" />'.format(url = new_url))